In [12]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
df_test = pd.read_csv('../data/test_values.csv')
df_train = pd.read_csv('../data/train_values.csv')
df_label = pd.read_csv('../data/train_labels.csv')

In [ ]:
#df_test.info()
#df_train.info()
df_label.info()
df_label.head()


In [17]:
import encoding_category as ec
train_ec, test_ec = ec.encode_cat(df_train, df_test)


AttributeError: module 'encoding_category' has no attribute 'encode_cat'